In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment = Experiment(ws, 'Bank_Marketing_AutoML')
print("Experiment created")

print('WORKSPACE DETAILS:\nWorkspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Experiment created
WORKSPACE DETAILS:
Workspace name: quick-starts-ws-134068
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-134068


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###

cpu_cluster_name = "cpu-cluster"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except ComputeTargetException:
    print("Creating new Compute Target... " + cpu_cluster_name)
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Existing compute target found... Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [4]:
# checking the uploaded data
ds_dataframe = ds.to_pandas_dataframe()
ds_dataframe.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [5]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [6]:
# saving cleaned data locally

import os
import pandas as pd
path = "./Data"
try:
    os.makedirs(path, exist_ok=True)
    print("Dicrectory '%s' created...")
except OSError:
    print("Directory '%s' cannot be created...")


Dicrectory '%s' created...


In [7]:
clean_df = x.copy(deep=True)
clean_df['y'] = y
clean_df.to_csv('./Data/clean_data.csv')
print(x.shape, y.shape, clean_df.shape)
print(clean_df.head())

(32950, 39) (32950,) (32950, 40)
   age  marital  default  housing  loan  month  day_of_week  duration  \
0   57        1        0        0     1      5            1       371   
1   55        1        0        1     0      5            4       285   
2   33        1        0        0     0      5            5        52   
3   36        1        0        0     0      6            5       355   
4   27        1        0        1     0      7            5       189   

   campaign  pdays  ...  contact_telephone  education_basic.4y  \
0         1    999  ...                  0                   0   
1         2    999  ...                  1                   0   
2         1    999  ...                  0                   0   
3         4    999  ...                  1                   0   
4         2    999  ...                  0                   0   

   education_basic.6y  education_basic.9y  education_high.school  \
0                   0                   0                      

In [8]:
# split data into test and train sets

from sklearn.model_selection import train_test_split
import pandas as pd

#spliting the data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

# copying series y column to the train data
train_df = train_x.copy(deep=True)
train_df['y'] = train_y
print(train_x.shape, train_y.shape)

# joining series y column to the test data
test_df = test_x.copy(deep=True)
test_df['y'] = test_y
print(test_x.shape, test_y.shape)

print(train_df.shape)
print(test_df.shape)

# saving test and train data locally
train_df.to_csv("./Data/train_data.csv")
test_df.to_csv("./Data/test_data.csv")

(23065, 39) (23065,)
(9885, 39) (9885,)
(23065, 40)
(9885, 40)


In [9]:
# upload the Data directory to the datastore

deafult_store = ws.get_default_datastore()
deafult_store.upload(src_dir="./Data", target_path="./Data", overwrite=True)

Uploading an estimated of 3 files
Uploading ./Data/test_data.csv
Uploaded ./Data/test_data.csv, 1 files out of an estimated total of 3
Uploading ./Data/clean_data.csv
Uploaded ./Data/clean_data.csv, 2 files out of an estimated total of 3
Uploading ./Data/train_data.csv
Uploaded ./Data/train_data.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_d98149be4bfb479eaeeef8cd5d25fe0e

In [10]:
# Convert the DataFrame to TabularDataset
from azureml.core import Dataset

clean_data = Dataset.Tabular.from_delimited_files(path=[(deafult_store, ("./Data/clean_data.csv"))])
train_data = Dataset.Tabular.from_delimited_files(path=[(deafult_store, ("./Data/train_data.csv"))])
test_data = Dataset.Tabular.from_delimited_files(path=[(deafult_store, ("./Data/test_data.csv"))])

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5,
    compute_target = cpu_cluster)

print("AutoML config created.")

AutoML config created.


In [13]:
# Submit your automl run
from azureml.widgets import RunDetails

### YOUR CODE HERE ###
automl_run = experiment.submit(automl_config, show_output = True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_6a134e7a-5f4f-44cb-a5b8-f187191ddf74

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+-------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|2561                             |1                                |23065                                 |
+---------------------------------+----------

{'runId': 'AutoML_6a134e7a-5f4f-44cb-a5b8-f187191ddf74',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-09T10:28:06.901768Z',
 'endTimeUtc': '2021-01-09T11:09:43.351616Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Bank_Marketing_AutoML","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-134068","workspace_name":"quick-starts-ws-134068","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes"

In [14]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: Bank_Marketing_AutoML,
Id: AutoML_6a134e7a-5f4f-44cb-a5b8-f187191ddf74_27,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_leaf=0.01,
                                                                                                    min_samples_split=0.01,
          

In [15]:
best_run.register_model(model_name="Bank_Marketing_AutoML_best_model.pkl", model_path ="./outputs/")

Model(workspace=Workspace.create(name='quick-starts-ws-134068', subscription_id='b968fb36-f06a-4c76-a15f-afab68ae7667', resource_group='aml-quickstarts-134068'), name=Bank_Marketing_AutoML_best_model.pkl, id=Bank_Marketing_AutoML_best_model.pkl:1, version=1, tags={}, properties={})

In [16]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":1,"CurrentNodeCount":1}',
 'ensembled_iterations': '[1, 0, 20, 23, 10, 5, 26, 7, 4]',
 'ensembled_algorithms': "['XGBoostClassifier', 'LightGBM', 'RandomForest', 'LogisticRegression', 'LogisticRegression', 'XGBoostClassifier', 'LightGBM', 'XGBoostClassifier', 'RandomForest']",
 'ensemble_weights': '[0.2, 0.2, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.2]',
 'best_individual_pipeline_score': '0.9164968567092998',
 'best_individual_iteration': '1',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

In [17]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Bank_Marketing_AutoML,AutoML_6a134e7a-5f4f-44cb-a5b8-f187191ddf74_27,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation
